In [1]:
# ETL trinity data
# Conny Lin | June 4, 2020

# prepare the data into machine learning ready format.
# Extract data from individual plates, label with plate index in the plate db, and group name.

In [2]:
# load library
import os
import pickle
import numpy as np
import pandas as pd
import subprocess
import glob

In [3]:
# define global varialbes/paths
dir_save = '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone/data'

# make data db

In [5]:
# load file pickle
plateDB = pickle.load(open(os.path.join(dir_save, 'file_summary_mwt.pickle'),'rb'))
# get paths with trinity.id.dat
pMWT = plateDB.index[~plateDB[('filepath','trinity.id.dat')].isna()].values
del plateDB

In [27]:
# make dataframe
MWTDB = pd.DataFrame({'mwtpath':pMWT})

In [30]:
# take a look at the db to see if any missing trinity pickle 
# instantiate 
report_capture = np.zeros(len(pMWT),dtype='bool')
for plateid, pPlate in enumerate(MWTDB['mwtpath']):
    # get expected apth to trinity data
    pfile = os.path.join(pPlate, 'trinity_all_worms.pickle')
    # see if file exist
    if os.path.exists(pfile):
        report_capture[plateid] = True
    else:
        print(f'{plateid} does not exist', end='\r')

# report result
print(f'{np.sum(report_capture)}/{len(report_capture)} files exist')

870/871 files exist


In [34]:
# delete the plate that failed to concatenate trinity
MWTDB.drop(index=MWTDB.index[~report_capture].values, inplace=True)

# Create MWTDB

In [42]:
# add paths to trinitu files
MWTDB['trinity_path'] = list(map(lambda x: os.path.join(x,'trinity_all_worms.pickle'), MWTDB['mwtpath']))

In [62]:
# reset index
MWTDB.reset_index(drop=True, inplace=True)

In [85]:
# extract experiment features
df = MWTDB['mwtpath'].str.split(pat='/', expand=True)
MWTDB['expname'] = df.iloc[:,4]
MWTDB['groupname'] = df.iloc[:,5]
MWTDB['platename'] = df.iloc[:,6]

In [86]:
# save database
pickle.dump(MWTDB, open(os.path.join(dir_save, 'MWTDB_trinity_N2400mM.pickle'),'wb'))

# Data wrangling

## add labels to individual plate data

In [109]:
# take a sample to see if need per file processing
for ind in MWTDB.index.values:
    # get path
    ptrinity = MWTDB['trinity_path'].iloc[ind]
    # load to dataframe
    df = pickle.load(open(ptrinity,'rb'))
    row_n_original = df.shape[0]
    # check if the data already been cleaned
    if any(df.columns=='mwtid_trdb'):
        continue
    # clean nan data
    df.dropna(axis=0, inplace=True)
    row_n_after = df.shape[0]
    print(f'plateid {ind} dropped {row_n_original - row_n_after} rows to {row_n_after} rows', end='\r')
    # add file path 
    df.insert(0,'mwtid_trdb', np.tile(MWTDB.index[ind], df.shape[0]))
    # add group id (ethanol=1 vs no ethanol=0)
    if MWTDB['groupname'][ind]=='N2':
        df.insert(1,'etoh', np.tile(0, df.shape[0]))
    else:
        df.insert(1,'etoh', np.tile(1, df.shape[0]))
    # save the file
    pickle.dump(df, open(ptrinity,'wb'))
    

## concat all trinity data
https://stackoverflow.com/questions/56012595/how-to-pickle-multiple-pandas-dataframes-and-concatenate-all-of-them-i

```
df = pd.concat([pd.read_pickle('/PATH/df/{}/{}.F.K.df'.format('train', f)).iloc[:, :100] 
                for f in Files], 
               axis=1)
```

`a = [pd.read_pickle(p) for p in MWTDB['trinity_path'][:10]]`

Issues:

* Each csv is ~100MB * 800 = 80GB csv. My computer won't be able to open this file. 


In [ ]:
# look at  behavior and see if can predict which tap it is